In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
a_questions = pd.DataFrame.from_csv('question_simple.csv', index_col=None)
a_questions[a_questions.AcceptedAnsId.notnull()].head()

,QuestionId,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate
0,1,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000
1,2,2011-11-15T20:50:11.270,2011-12-02T11:38:48.620,10,2011-11-16T00:00:00.000
2,3,2011-11-15T20:52:27.287,2015-07-18T09:49:54.953,109,2013-01-19T00:00:00.000
3,7,2011-11-15T20:56:40.383,2015-11-23T12:25:29.887,2977,2012-10-22T00:00:00.000
4,14,2011-11-15T21:01:05.117,2015-08-13T13:14:56.870,20,2011-11-16T00:00:00.000


In [3]:
a_votes = pd.DataFrame.from_csv('votes-answers.csv', index_col=None)
a_votes = pd.merge(a_votes, a_questions, how='inner', on=['QuestionId'],suffixes=['_vot', '_quest'])
a_votes['AcceptedDate'].fillna(pd.to_datetime('20160320'),inplace=True) #Today
a_votes.head()

,VoteCreation,AnsCreation,VoteType,AnsId,QuestionId,AnsWordCount,QuestionCreation,QuestionLastActivity,AcceptedAnsId,AcceptedDate
0,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000
1,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,2,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000
2,2011-11-15T00:00:00.000,2011-11-15T20:54:09.083,2,5,1,37,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000
3,2011-11-15T00:00:00.000,2011-11-15T20:53:36.853,3,4,1,190,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000
4,2011-11-15T00:00:00.000,2011-11-15T21:34:26.457,2,56,1,201,2011-11-15T20:45:51.037,2015-10-21T09:57:50.000,56,2011-11-21T00:00:00.000


In [4]:
a_votes['AcceptedAge'] = (pd.to_datetime(a_votes.AcceptedDate,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['AcceptedAge'] = a_votes['AcceptedAge'] + 1

a_votes.loc[a_votes.AcceptedDate == pd.to_datetime('20160320'), 'AcceptedAge'] = -1

a_votes['RelAge'] = (pd.to_datetime(a_votes.VoteCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.AnsCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['RelAge'] = a_votes['RelAge'] + 1

a_votes['Age'] = (pd.to_datetime(a_votes.VoteCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_votes.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_votes['Age'] = a_votes['Age'] + 1
a_votes.drop(a_votes.columns[[0, 1, 6, 7,9]], axis=1, inplace=True)
a_votes.head(20)

,VoteType,AnsId,QuestionId,AnsWordCount,AcceptedAnsId,AcceptedAge,RelAge,Age
0,2,4,1,190,56,6,0,0
1,2,4,1,190,56,6,0,0
2,2,5,1,37,56,6,0,0
3,3,4,1,190,56,6,0,0
4,2,56,1,201,56,6,0,0
5,2,56,1,201,56,6,0,0
6,2,56,1,201,56,6,1,1
7,2,56,1,201,56,6,1,1
8,2,56,1,201,56,6,1,1
9,2,210,1,40,56,6,0,1


In [5]:
a_comments = pd.DataFrame.from_csv('comment-ans.csv', index_col=None)
a_comments = pd.merge(a_comments, a_questions, how='inner', on=['QuestionId'],suffixes=['_vot', '_quest'])

a_comments['Age'] = (pd.to_datetime(a_comments.CommentCreation,format='%Y-%m-%d %H:%M:%S.%f')
                  -pd.to_datetime(a_comments.QuestionCreation,format='%Y-%m-%d %H:%M:%S.%f')).apply(lambda x: x.astype('timedelta64[D]').item().days)
a_comments['Age'] = a_comments['Age'] + 1
a_comments = a_comments[['AnsId','Age']]
a_comments.head(10)

,AnsId,Age
0,4,1
1,4,1
2,5,1
3,56,1
4,56,1
5,56,1
6,56,1
7,7030,951
8,7030,1285
9,15104,1436


In [6]:
get_score = lambda df: sum(df.VoteType==2) - sum(df.VoteType==3)
get_votes = lambda df: sum(df.VoteType==2) + sum(df.VoteType==3)
f = lambda df: pd.Series([df.AnsWordCount.iloc[0],df.AcceptedAnsId.iloc[0],df.AcceptedAge.iloc[0],get_score(df),get_votes(df)],index=['AnsWordCount','AcceptedAnsId','AcceptedAge','Score','Votes'])
a_groups = a_votes.sort_values(by='Age').groupby(['QuestionId','AnsId','Age']).apply(f)
a_groups = a_groups.reset_index(level=[0,1,2],drop=False)
a_groups.head(15)

,QuestionId,AnsId,Age,AnsWordCount,AcceptedAnsId,AcceptedAge,Score,Votes
0,1,4,0,190,56,6,1,3
1,1,4,1436,190,56,6,1,1
2,1,5,0,37,56,6,1,1
3,1,5,108,37,56,6,-1,1
4,1,56,0,201,56,6,2,2
5,1,56,1,201,56,6,3,3
6,1,56,4,201,56,6,1,1
7,1,56,7,201,56,6,1,1
8,1,56,157,201,56,6,1,1
9,1,56,905,201,56,6,1,1


In [8]:
f =lambda df: pd.concat([df['AnsId'],df['QuestionId'],df['Age'],df['AnsWordCount'],df['AcceptedAnsId'],df['AcceptedAge'],df['Votes'],df['Score'].cumsum()],axis=1)
a_groups_c = a_groups.sort_values(by='Age').groupby(['AnsId']).apply(f)
a_groups_c.head(15)

,AnsId,QuestionId,Age,AnsWordCount,AcceptedAnsId,AcceptedAge,Votes,Score
0,4,1,0,190,56,6,3,1
10437,5402,5400,0,81,5402,0,1,1
10435,5399,5398,0,70,5399,0,1,1
2643,813,812,0,381,813,6,6,6
10434,5410,5395,0,56,5396,0,1,1
10433,5405,5395,0,153,5396,0,3,3
10432,5397,5395,0,47,5396,0,2,2
10431,5396,5395,0,298,5396,0,2,2
10428,5394,5393,0,17,5394,55,4,4
10422,5392,5391,0,11,5392,0,3,3


In [10]:
votes_com = pd.merge(a_groups_c, a_comments, how='left', on=['AnsId'],suffixes=['_vot', '_com'])
f = lambda df:pd.concat([df['AnsId'],df['QuestionId'],df['Age_vot'],df['AnsWordCount'],df['AcceptedAnsId'],df['AcceptedAge'],df['Votes'],df['Score'],pd.Series(df['Age_vot']>=df['Age_com'],name='gt')],axis=1)
votes_com = votes_com.groupby(['AnsId']).apply(f)
#votes_com.columns.values[5]='gt'
votes_com.head(10)

,AnsId,QuestionId,Age_vot,AnsWordCount,AcceptedAnsId,AcceptedAge,Votes,Score,gt
0,4,1,0,190,56,6,3,1,False
1,4,1,0,190,56,6,3,1,False
2,5402,5400,0,81,5402,0,1,1,False
3,5402,5400,0,81,5402,0,1,1,False
4,5402,5400,0,81,5402,0,1,1,False
5,5402,5400,0,81,5402,0,1,1,False
6,5399,5398,0,70,5399,0,1,1,False
7,813,812,0,381,813,6,6,6,False
8,813,812,0,381,813,6,6,6,False
9,813,812,0,381,813,6,6,6,False


In [11]:
tmp_sum_com = votes_com.groupby(['AnsId','Age_vot']).apply(lambda df: sum(df['gt']))
tmp_sum_com.reset_index(drop=False).head(8)

,AnsId,Age_vot,0
0,4,0,0
1,4,1436,2
2,5,0,0
3,5,108,1
4,6,0,0
5,8,0,0
6,9,0,0
7,9,12,3


## First Time series cleaned (model for the number of votes)

In [74]:
votes_com_f = pd.merge(votes_com,tmp_sum_com.reset_index(drop=False),on=['AnsId','Age_vot'],how='inner')
votes_com_f.drop(votes_com_f.columns[[8]], axis=1, inplace=True)
votes_com_f.columns.values[8]='Comments'
votes_com_f = votes_com_f.drop_duplicates()
votes_com_f.head(20)

,AnsId,QuestionId,Age_vot,AnsWordCount,AcceptedAnsId,AcceptedAge,Votes,Score,Comments
0,4,1,0,190,56,6,3,1,0
2,5402,5400,0,81,5402,0,1,1,0
6,5399,5398,0,70,5399,0,1,1,0
7,813,812,0,381,813,6,6,6,0
11,5410,5395,0,56,5396,0,1,1,0
12,5405,5395,0,153,5396,0,3,3,0
13,5397,5395,0,47,5396,0,2,2,0
14,5396,5395,0,298,5396,0,2,2,0
16,5394,5393,0,17,5394,55,4,4,0
17,5392,5391,0,11,5392,0,3,3,0


# Sanity check

In [75]:
votes_com_f[votes_com_f['AnsId'].isnull() | votes_com_f['Age_vot'].isnull() | votes_com_f['Votes'].isnull() | votes_com_f['Score'].isnull() | votes_com_f['Comments'].isnull()]

,AnsId,QuestionId,Age_vot,AnsWordCount,AcceptedAnsId,AcceptedAge,Votes,Score,Comments


In [13]:
# A few auxiliar functions
from itertools import izip
def rank_ans(df):
    def rank_iter():
        cache = {}
        accepted = 0
        for row in df.itertuples():
            cache[row.AnsId] = row.Score
            # rank, nb_ans
            if row.AcceptedAge>-1 and (row.AnsId == row.AcceptedAnsId) and row.Age_vot >=row.AcceptedAge:
                accepted = 1
                if row.AnsId in cache:
                    del cache[row.AnsId]
                yield (1,len(cache)+1,row.Index)
            else :
                rank = sorted(cache, key= lambda k:cache[k],reverse=True).index(row.AnsId) + 1 + accepted
                yield (rank,len(cache),row.Index)
            
    ranks, ans_counts, indices = izip(*list(rank_iter())) #TODO: optimize for the future
    return [pd.Series(ranks,name="Ans_rank", index=indices), pd.Series(ans_counts,name="Ans_count", index=indices)]

get_ranks = lambda df: pd.concat([df['AnsId'],df['Age_vot'],df['Votes'],df['Score'],df['Comments'],df['AnsWordCount']] + rank_ans(df),axis=1)
sort_age_score = lambda df: df.sort_values(by=['Age_vot','Score'],ascending=[True,False])

## Base data for model

In [76]:
votes_com_f = votes_com_f.groupby(['QuestionId']).apply(lambda df: get_ranks(sort_age_score(df)))
votes_com_f.head(20)

AnsId  Age_vot  Votes  Score  Comments  AnsWordCount  \
QuestionId                                                               
1          5662      56        0      2      2         0           201   
           0          4        0      3      1         0           190   
           5649       5        0      1      1         0            37   
           11654     56        1      3      5         4           201   
           11568    210        1      1      1         0            40   
           16847     56        4      1      6         4           201   
           16983    210        4      1      2         0            40   
           20112     56        7      1      7         4           201   
           26975    897      108      1      1         0            68   
           26948      5      108      1      0         1            37   
           27785     56      157      1      8         4           201   
           34032     56      905      1      9         4           201   
           34367   7030      951      1      1         1           145   
           35932   7030     1231      1      2         1           145   
           36092   7030     1255      1      3         1           145   
           36235   7030     1284      1      4         1           145   
           36960   7030     1400      1      5         2           145   
           36986   7030     1405      1      6         2           145   
           37125      4     1436      1      2         2           190   
           37124  15104     1436      3     -3         1           108   

                  Ans_rank  Ans_count  
QuestionId                             
1          5662          1          1  
           0             2          2  
           5649          3          3  
           11654         1          3  
           11568         2          4  
           16847         1          4  
           16983         2          4  
           20112         1          4  
           26975         3          4  
           26948         5          4  
           27785         1          5  
           34032         1          5  
           34367         5          5  
           35932         3          5  
           36092         2          5  
           36235         2          5  
           36960         2          5  
           36986         2          5  
           37125         3          5  
           37124         7          6

In [77]:
votes = votes_com_f.reset_index(level=[0],drop=False)
votes.describe()

,QuestionId,AnsId,Age_vot,Votes,Score,Comments,AnsWordCount,Ans_rank,Ans_count
count,15779.000000,15779.000000,15779.000000,15779.000000,15779.000000,15779.000000,15779.000000,15779.000000,15779.000000
mean,5262.750808,5546.051904,177.569301,1.439255,4.449965,1.225299,126.761011,1.599721,2.254832
std,4757.595854,4792.079607,351.781626,1.007782,4.957413,2.072676,129.504190,1.027504,1.597277
min,1.000000,4.000000,0.000000,1.000000,-7.000000,0.000000,3.000000,1.000000,1.000000
25%,1474.000000,1725.500000,1.000000,1.000000,2.000000,0.000000,46.000000,1.000000,1.000000
50%,3558.000000,3873.000000,5.000000,1.000000,3.000000,0.000000,89.000000,1.000000,2.000000
75%,8569.500000,9846.000000,120.000000,1.000000,6.000000,2.000000,160.000000,2.000000,3.000000
max,15699.000000,15701.000000,1504.000000,12.000000,56.000000,18.000000,2142.000000,11.000000,12.000000


In [69]:
#by_ans = pd.get_dummies(votes.AnsId)
#by_question = pd.get_dummies(votes.QuestionId)
#votes = pd.concat([votes,by_ans,by_question],axis=1)
#votes.head()

In [71]:
votes['Age_vot'].value_counts().head(15)

0    2659
1    2512
2    1228
3     806
4     539
Name: Age_vot, dtype: int64

In [78]:
votes['ReScore'] = votes['Score']/(votes['Votes']+1.0)
#votes['TScore'] = np.power(votes['Score'],votes['age_vot'])
votes['Pbias'] = 1.0/votes['Ans_rank']
#votes = votes[ votes.age_vot>=3]
votes = votes[votes.Age_vot<=4]
votes = votes[votes.Ans_rank<=5]
votes[['Score','Votes','Ans_rank','Ans_count']].describe()

,Score,Votes,Ans_rank,Ans_count
count,7787.000000,7787.000000,7787.000000,7787.000000
mean,2.904071,1.807114,1.410171,1.638885
std,2.296407,1.295147,0.717825,0.934576
min,-6.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000
50%,2.000000,1.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000,2.000000
max,17.000000,12.000000,5.000000,9.000000


In [18]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [20]:
print "Pearson r Score-position: ", votes['Score'].corr(votes['Ans_rank'],method='pearson')

Pearson r Score-position:  -0.334287781557


## Mixed effects model for votes

In [79]:
formula = "np.log(Votes+1) ~ C(Ans_rank) - 1"
model = sm.MixedLM.from_formula(formula, votes, re_formula="Ans_rank", groups=votes["AnsId"])
result = model.fit()
print result.summary()

                 Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   np.log(Votes + 1)
No. Observations:    7787      Method:               REML             
No. Groups:          4582      Scale:                0.1239           
Min. group size:     1         Likelihood:           -2978.2759       
Max. group size:     5         Converged:            Yes              
Mean group size:     1.7                                              
----------------------------------------------------------------------
                           Coef.  Std.Err.    z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
C(Ans_rank)[1]              1.003    0.005 205.445 0.000  0.993  1.013
C(Ans_rank)[2]              0.867    0.008 102.366 0.000  0.851  0.884
C(Ans_rank)[3]              0.811    0.016  49.601 0.000  0.779  0.843
C(Ans_rank)[4]              0.799    0.032  24.673 0.000  0.736  0.863
C(Ans_rank)[5]        

In [80]:
formula = "np.log(Votes+1) ~ C(Ans_rank,Helmert) - 1"
model = sm.MixedLM.from_formula(formula, votes, re_formula="Ans_rank", groups=votes["AnsId"])
result = model.fit()
print result.summary()

                    Mixed Linear Model Regression Results
Model:                  MixedLM     Dependent Variable:     np.log(Votes + 1)
No. Observations:       7787        Method:                 REML             
No. Groups:             4582        Scale:                  0.1239           
Min. group size:        1           Likelihood:             -2983.0634       
Max. group size:        5           Converged:              Yes              
Mean group size:        1.7                                                  
-----------------------------------------------------------------------------
                                  Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------
C(Ans_rank, Helmert)[H.intercept]  0.849    0.014  62.128 0.000  0.822  0.876
C(Ans_rank, Helmert)[H.2]         -0.068    0.005 -13.881 0.000 -0.077 -0.058
C(Ans_rank, Helmert)[H.3]         -0.041    0.006  -7.275 0.000 -0.053 -0.030
C(Ans_

In [81]:
formula = "np.log(Votes+1) ~ C(Ans_rank,Diff) - 1"
model = sm.MixedLM.from_formula(formula, votes, re_formula="Ans_rank", groups=votes["AnsId"])
result = model.fit()
print result.summary()

                 Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   np.log(Votes + 1)
No. Observations:    7787      Method:               REML             
No. Groups:          4582      Scale:                0.1239           
Min. group size:     1         Likelihood:           -2978.2761       
Max. group size:     5         Converged:            Yes              
Mean group size:     1.7                                              
----------------------------------------------------------------------
                           Coef.  Std.Err.    z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
C(Ans_rank, Diff)[D.1]      0.849    0.014  62.116 0.000  0.822  0.876
C(Ans_rank, Diff)[D.2]     -0.136    0.010 -13.882 0.000 -0.155 -0.117
C(Ans_rank, Diff)[D.3]     -0.056    0.018  -3.052 0.002 -0.092 -0.020
C(Ans_rank, Diff)[D.4]     -0.012    0.036  -0.318 0.750 -0.083  0.060
C(Ans_rank, Diff)[D.5]

## OLS, Poisson, RobustLM

In [34]:
mod_ols = smf.ols(formula='Votes ~ ReScore*Pbias*(AnsWordCount + Score + Comments  + Ans_count) -1 ', data=votes)
res = mod_ols.fit()
print res.summary()

                            OLS Regression Results                            
Dep. Variable:                  Votes   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     9074.
Date:                Sun, 20 Mar 2016   Prob (F-statistic):               0.00
Time:                        06:55:04   Log-Likelihood:                -6255.8
No. Observations:                9364   AIC:                         1.255e+04
Df Residuals:                    9345   BIC:                         1.269e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------------
ReScore             

In [35]:
mod_poi = smf.glm('Votes ~ Score + Ans_rank + AnsWordCount + Comments  + Ans_count - 1 ', data=votes,
                  family=sm.families.Poisson(link=sm.families.links.log)).fit(method='bfgs') #
#print mod_poi.mle_retvals['converged']
print mod_poi.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Votes   No. Observations:                 9364
Model:                            GLM   Df Residuals:                     9359
Model Family:                 Poisson   Df Model:                            4
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -13488.
Date:                Sun, 20 Mar 2016   Deviance:                       6748.6
Time:                        06:55:20   Pearson chi2:                 6.14e+03
No. Iterations:                     8                                         
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Score            0.1245      0.003     48.930      0.000         0.120     0.130
Ans_rank         0.1981      0.011     17.433 

In [19]:
huber_t = sm.RLM(np.log(votes.Votes + 1), votes[['age_vot','Score', 'Comments','Ans_rank','Ans_count']], M=sm.robust.norms.HuberT())
hub_results = huber_t.fit()

#print hub_results.params
#print hub_results.bse
#varnames = ['var_%d' % i for i in range(len(hub_results.params))]
print hub_results.summary()

                    Robust linear Model Regression Results                    
Dep. Variable:                  Votes   No. Observations:                 2669
Model:                            RLM   Df Residuals:                     2664
Method:                          IRLS   Df Model:                            4
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Thu, 17 Mar 2016                                         
Time:                        04:50:12                                         
No. Iterations:                    25                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
age_vot        0.0285      0.001     23.031      0.0

In [191]:
ii = (e for e in range(5))
list(ii)

[0, 1, 2, 3, 4]